In [ ]:
import collections

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from sklearn.cluster import SpectralClustering
from sklearn.manifold import TSNE
from torch import Tensor
from transformers import AutoModel, AutoTokenizer

In [ ]:
train_data = pd.read_csv("train.csv")
headlines = train_data["text"].tolist()

In [ ]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f"Instruct: {task_description}\nQuery: {query}"


# Each query must come with a one-sentence instruction that describes the task
task = "Given a Korean news headline, retrieve other news headlines that discuss similar topics or events."

input_texts = [
    get_detailed_instruct(task, headline)
    for headline in headlines  # headlines는 뉴스 헤드라인 리스트
]

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct").to("cuda")

batch_size = 64  # 배치 크기 설정
all_embeddings = []

for i in range(0, len(input_texts), batch_size):
    # 배치 단위로 토큰화
    batch_input_texts = input_texts[i:i + batch_size]
    batch_dict = tokenizer(
        batch_input_texts,
        max_length=512,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to("cuda")

    # 모델로부터 임베딩 생성 및 평균 풀링
    with torch.no_grad():
        outputs = model(**batch_dict)
        batch_embeddings = average_pool(
            outputs.last_hidden_state, batch_dict["attention_mask"]
        )
        batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
        all_embeddings.append(batch_embeddings.cpu())

# 전체 임베딩 텐서로 결합
embeddings = torch.cat(all_embeddings)

scores = embeddings @ embeddings.T
scores.fill_diagonal_(-1)  # query인 문장은 유사도 계산에서 제외
train_data["embedding"] = embeddings.cpu().tolist()

In [ ]:
# 추가할 열 이름들
new_columns = ["0", "1", "2", "3", "4", "5", "6"]

# 각 열에 대해 0으로 채운 값 추가
for col in new_columns:
    train_data[col] = 0

# 결과 확인
train_data

In [ ]:
# 유사한 문장 중 공통된 label을 뽑는 과정
# 두 가지 접근법
# 1. query와 관련된 상위 50개 문장의 label 중 가장 많은 클래스로 label을 수정하는 방식 (top-k_vote 열에 기록)
# 2. 각 문장이 어떠한 label로 얼마나 많이 retrieve 되었는지를 세어서 가장 많이 사용된 label로 수정하는 방식 (max_0-6 열에 기록)

for i in range(0, len(train_data)):
    vote = []
    top_scores, top_indices = scores[i].topk(50)
    print(
        f"헤드라인 {i}와 유사한 상위 50개 헤드라인:",
        headlines[i],
        train_data["target"][i],
    )
    for score, idx in zip(top_scores, top_indices):
        print(
            f"  - 헤드라인 {idx}: 유사도 {score.item()}",
            headlines[idx],
            train_data["target"][idx.item()],
        )
        vote.append(train_data["target"][idx.item()])
    count = collections.Counter(vote)
    train_data.loc[i, "top-k_vote"] = max(count, key=count.get)
    print(count)
    for score, idx in zip(top_scores, top_indices):
        train_data.loc[idx.item(), str(max(count, key=count.get))] += 1
train_data["max_0-6"] = train_data.loc[:, "0":"6"].idxmax(axis=1)
train_data["top-k_vote"] = train_data["top-k_vote"].astype(int)
train_data["max_0-6"] = train_data["max_0-6"].astype(int)
train_data["same"] = train_data["top-k_vote"] == train_data["max_0-6"]

In [ ]:
train_data

In [ ]:
np.random.seed(42)

# threshold 설정
threshold = np.percentile(scores, 80)
# 임계값보다 낮은 유사도는 0으로 설정
scores[scores < threshold] = 0

# 스펙트럴 클러스터링 수행
n_clusters = 7  # 원하는 군집 수
spectral_cluster = SpectralClustering(
    n_clusters=n_clusters, affinity="precomputed", random_state=42
)
clusters = spectral_cluster.fit_predict(scores)

In [ ]:
train_data["SpectralClustering"] = clusters

# 매핑 딕셔너리 생성
# SpectralClustering하여 얻은 클래스 번호가 train data의 클래스 번호가 다르기 때문에 시각화 및 비교를 하기 위해 매핑
mapping = {0: 5, 1: 2, 2: 6, 3: 1, 4: 3, 5: 4, 6: 0}  # 원하는 대로 값 매핑
train_data["SpectralClustering_mapping"] = train_data["SpectralClustering"].map(mapping)

In [ ]:
# 임베딩 데이터를 배열로 변환
embeddings = np.array(train_data["embedding"].tolist())

# t-SNE를 통해 2차원으로 차원 축소
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(embeddings)

# 차원 축소된 데이터를 데이터프레임에 추가
train_data["tsne_2d_x"] = embeddings_2d[:, 0]
train_data["tsne_2d_y"] = embeddings_2d[:, 1]


# 서브플롯 생성
fig, axes = plt.subplots(2, 2, figsize=(20, 20))

# 첫 번째 시각화 - train data의 target
sns.scatterplot(
    x="tsne_2d_x",
    y="tsne_2d_y",
    hue="target",  # target 열을 기준으로 색상 지정
    palette=sns.color_palette("hsv", len(train_data["target"].unique())),
    data=train_data,
    legend="full",
    alpha=0.7,
    ax=axes[0, 0],
)
axes[0, 0].set_title("t-SNE Visualization by Target Label")
axes[0, 0].set_xlabel("t-SNE Dimension 1")
axes[0, 0].set_ylabel("t-SNE Dimension 2")
axes[0, 0].legend(loc="best")

# 두 번째 시각화 - Spectral Clustering
sns.scatterplot(
    x="tsne_2d_x",
    y="tsne_2d_y",
    hue="SpectralClustering_mapping",  # SpectralClustering_mapping 열을 기준으로 색상 지정
    palette=sns.color_palette(
        "hsv", len(train_data["SpectralClustering_mapping"].unique())
    ),
    data=train_data,
    legend="full",
    alpha=0.7,
    ax=axes[0, 1],
)
axes[0, 1].set_title("t-SNE Visualization of by Spectral Clustering")
axes[0, 1].set_xlabel("t-SNE Dimension 1")
axes[0, 1].set_ylabel("t-SNE Dimension 2")
axes[0, 1].legend(loc="best")

# 세 번째 시각화 - query와 관련된 상위 50개 문장의 label 중 가장 많은 클래스로 label을 수정하는 방식
sns.scatterplot(
    x="tsne_2d_x",
    y="tsne_2d_y",
    hue="top-k_vote",  # top-k_vote 열을 기준으로 색상 지정
    palette=sns.color_palette("hsv", len(train_data["top-k_vote"].unique())),
    data=train_data,
    legend="full",
    alpha=0.7,
    ax=axes[1, 0],
)
axes[1, 0].set_title("t-SNE Visualization by Majority Label in Top-50 Retrievals")
axes[1, 0].set_xlabel("t-SNE Dimension 1")
axes[1, 0].set_ylabel("t-SNE Dimension 2")
axes[1, 0].legend(loc="best")

# 네 번째 시각화 - 각 문장이 어떠한 label로 얼마나 많이 retrieve 되었는지를 세어서 가장 많이 사용된 label로 수정하는 방식
sns.scatterplot(
    x="tsne_2d_x",
    y="tsne_2d_y",
    hue="max_0-6",  # max_0-6 열을 기준으로 색상 지정
    palette=sns.color_palette("hsv", len(train_data["max_0-6"].unique())),
    data=train_data,
    legend="full",
    alpha=0.7,
    ax=axes[1, 1],
)
axes[1, 1].set_title(
    "t-SNE Visualization by Aggregated Majority Voting Across Top-50 Retrievals"
)
axes[1, 1].set_xlabel("t-SNE Dimension 1")
axes[1, 1].set_ylabel("t-SNE Dimension 2")
axes[1, 1].legend(loc="best")

# 전체 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()

In [ ]:
train_data.to_csv("train_data_relabeled.csv", index=False, encoding="utf-8")